In [ ]:
i = 5
plot_bif(["bbr-"+str(i)+"-0-50-1000-2-aws-244"], p="y",t=1)

In [ ]:
for i in range(1,6):
    plot_bif(["reno-"+str(i)+"-0-50-1000-1-aws"], p="y",t=1)

In [ ]:
for i in range(1,6):
    plot_bif(["cubic-"+str(i)+"-0-50-1000-1-aws"], p="y",t=1)

In [ ]:
plot_bif(["cubic-1-0-50-1000-1-aws"], p="y",t=2)

In [ ]:
plot_bif(["cubic-1-0-50-1000-2-aws"], p="y",t=1)

In [ ]:
plot_bif(["cubic-1-0-50-1000-2-aws"], p="y",t=2)

In [ ]:
def plot_bif(files, p,t=1):
    for f in files:
        fs = f.split("-")
        # experiment change start 
        pre = int(fs[2])
        post = int(fs[3])
        # experiment change end
        rtt = float(((pre+post)*2))/1000
#         print(rtt)
        ax = 0
        if p == 'y':
            fig, ax = plt.subplots(1,1, figsize=(15,8))
        
        data, time, drops, retrans, loss_data = get_window(f, "y",t)
#         print(retrans)
        if p == 'y':
            plot_d(ax, time, data, "r", "Normal")
            plot_d(ax, time, loss_data, "g", "Normal")
#             for t in retrans :
#                 plt.axvline(x = t, color = 'b')
#         time, data = get_fft_smoothening(ax,f,0,rtt,0)
#         plot_d(ax,time,data,"b","FFT Smoothened" )
#         print(len(time), len(data))
#         time, data = smoothen(time, data, rtt)
#         print(len(time), len(data))
#         if p == 'y':
#             plot_d(ax, time, data, "r", "Smoothened")
#             plot_data = np.array(data)
#             plot_data -= min(data)
#             plot_data /= (max(data)-min(data))
#             plot_d(ax, time, plot_data, "g", "Smoothened")
#         fd_time, fd_data = get_fd(time, data)
#         plot_d(ax, time, data, "g")
#         grad_time, grad_data = get_grad(time, data)
#         smooth_grad_time, smooth_grad_data = smooth_grad(grad_time, grad_data)
#         if p == 'y':
# #             grad_plot_data += min(data)
#             plot_d(ax, grad_time, grad_data, "r", "Gradient")
# #             plot_d(ax, smooth_grad_time, smooth_grad_data, "b", "Smooth Gradient")
# #             start = math.floor(0.05*len(grad_time))
#             devi = statistics.pstdev(grad_data)
#             mean = statistics.mean(grad_data)
#             m = 0.60
#             plt.fill_between(grad_time, [mean + devi*m for i in range(len(grad_time))], [mean - devi*m for i in range(len(grad_time))], color='y')
#         grad_step_time, grad_step_data = get_step_grad(time, data, rtt)
#         plot_d(ax, grad_step_time, grad_step_data, "b", "Step Gradient")
#         ax.title.set_text(f)
    if p=='y':    
        ax.legend()
        plt.show()
#     return time, data, grad_time, grad_data, rtt
#     return time, data

In [ ]:
import matplotlib.pyplot as plt
from scipy.fft import rfft, rfftfreq
from scipy.fft import irfft
import numpy as np
import statistics
from statistics import mean, pstdev
import pandas as pd

def smoothen(time, data, rtt):
    # Smoothening 
    left = 0
    right = 0
    run_sum = 0
    avg_data = []
    new_time = []
    roll_time = time
    roll_data = data
#     while(time[right]-time[left] < float(rtt)/2):
#         avg_data.append(data[right])
#         new_time.append(time[right])
#         right+=1
#     roll_data = pd.DataFrame(data)
#     roll_time = pd.DataFrame(time)
#     roll_data =roll_data.rolling(10,center=True).mean().dropna()
#     roll_time =roll_time.rolling(10,center=True).mean().dropna()
#     roll_data = list(roll_data[0])
#     roll_time = list(roll_time[0])
    while right < len(roll_time):
        while(right < len(roll_time) and (roll_time[right]-roll_time[left] < 2*rtt)):
            run_sum+=roll_data[right]
            right+=1
        new_time.append(float(roll_time[right-1]+roll_time[left])/2)
        avg_data.append(float(run_sum)/(right-left))
        run_sum-=roll_data[left]
        left+=1
    #experiment change start
#     avg_data = pd.DataFrame(avg_data)
#     new_time = pd.DataFrame(new_time)
#     avg_data =avg_data.rolling(50,center=True).mean().dropna()
#     new_time =new_time.rolling(50, center=True).mean().dropna()
#     avg_data = list(avg_data[0])
#     new_time = list(new_time[0])
#     print(avg_data)
#     print(new_time)
    #experiment end
#     right-=1
#     prev = right
#     while(time[right] - time[prev] < float(rtt)/2):
#         prev-=1
#     prev+=1
#     for i in range(prev, right+1):
#         avg_data.append(data[i])
#         new_time.append(time[i])
#     print(len(data), len(avg_data), len(new_time))
#     avg_data = n_data.rolling(window=60, center=True).mean()
#     new_time = time
#     for i in range(30):
#         avg_data[0][i] = data[i]
#     for i in range(-1, -30, -1):
#         avg_data[0][len(data) + i] = data[len(data) + i]
#     ans_data = avg_data[0]
#     Smoothen function works well.
    return new_time, avg_data
#     return roll_time, roll_data

def get_fft(data):
    n = len(data)
    data_step = 0.002
    yf = rfft(data)
    xf = rfftfreq(n,data_step)
    return yf,xf

def get_fft_smoothening(ax,f,norm,rtt,p):
    rtt=rtt
    data, time = get_window(f, "n")

    yf, xf = get_fft(data)
    abs_yf = np.abs(yf)
#     print(len(abs_yf), len(xf))
    sort_abs_yf = sorted(abs_yf)
    
#     plt.title("FFT Decomposition")
#     plt.plot(xf, sort_abs_yf, color='b', alpha=0.5, lw=1)
#     plt.scatter(xf, sort_abs_yf, color='r', s=2)
#     plt.show()

    thresh  = (1/rtt)
#     print("RTT", rtt, "Freq ", thresh)
    thresh_ind = 0
    for i in range(len(xf)) :
        freq = xf[i]
        if(freq > thresh):
            thresh_ind = i
            break
#     print("Index ", thresh_ind)
#     print(xf[0:thresh_ind])
#     plt.title("Cleaned Frequencies")
#     plt.plot(xf[0:thresh_ind],np.abs(yf[0:thresh_ind]))
#     plt.show()

    yf_clean = yf
    yf_clean[thresh_ind+1:] = 0
    new_f_clean = irfft(yf_clean)
#     plt.title("Frequencies smaller than " + str(thresh))
#     plt.plot(time, data, color="b", alpha=0.5, label='original signal')
    start_len = len(time) - len(new_f_clean)

    plot_data = new_f_clean
    
    if norm == 1:
        max_data = max(new_f_clean)
        min_data = min(new_f_clean)
        plot_data = (new_f_clean - min_data)/(max_data - min_data)
    if p:
        ax.plot(time[start_len:], plot_data, 'k', label='FFT smoothening', linewidth=1.5)
#     plt.legend()
#     plt.show()
    plot_time = time[start_len:] 
    return plot_time, plot_data

def get_grad(time, data):
    new_data = []
#     new_data.append(0)
    for i in range(1, len(data)):
        # experiment change start 
        diff = time[i]-time[i-1]
        
        diff = max(0.000001,diff)
        # experiment end
#         diff=1
        new_data.append((data[i]-data[i-1])/(diff))
    new_time = time[1:]
    plot_data = np.array(new_data)
    max_data = max(new_data)
    min_data = min(new_data)
    new_data = np.array(new_data)
    plot_data = (new_data - min_data)/(max_data-min_data)
    return new_time, plot_data
def plot_d(ax, time, data, c, l):
    ax.plot(time, data, color=c, lw=2, label = l)

def smooth_grad(grad_time, grad_data):
    dt = pd.DataFrame(grad_data)
    tm = pd.DataFrame(grad_time)
    dt = list(dt.rolling(50, center=True).mean().dropna()[0])
    tm = list(tm.rolling(50, center=True).mean().dropna()[0])
    return tm, dt

def plot_one_bt(files, p):
    for f in files:
        fs = f.split("-")
        # experiment change start 
        pre = int(fs[2])
        post = int(fs[3])
        # experiment change end
        rtt = float(((pre+post)*2))/1000
#         print(rtt)
        ax = 0
        if p == 'y':
            fig, ax = plt.subplots(1,1, figsize=(15,8))
        
        time, data = get_fft_smoothening(ax,f,0,rtt,0)
#         plot_d(ax,time,data,"b","FFT Smoothened" )
#         print(len(time), len(data))
        time, data = smoothen(time, data, rtt)
#         print(len(time), len(data))
        if p == 'y':
#             plot_d(ax, time, data, "r", "Smoothened")
            plot_data = np.array(data)
            plot_data -= min(data)
            plot_data /= (max(data)-min(data))
            plot_d(ax, time, plot_data, "g", "Smoothened")
#         fd_time, fd_data = get_fd(time, data)
#         plot_d(ax, time, data, "g")
        grad_time, grad_data = get_grad(time, data)
#         smooth_grad_time, smooth_grad_data = smooth_grad(grad_time, grad_data)
        if p == 'y':
#             grad_plot_data += min(data)
            plot_d(ax, grad_time, grad_data, "r", "Gradient")
#             plot_d(ax, smooth_grad_time, smooth_grad_data, "b", "Smooth Gradient")
#             start = math.floor(0.05*len(grad_time))
            devi = statistics.pstdev(grad_data)
            mean = statistics.mean(grad_data)
            m = 0.60
            plt.fill_between(grad_time, [mean + devi*m for i in range(len(grad_time))], [mean - devi*m for i in range(len(grad_time))], color='y')
#         grad_step_time, grad_step_data = get_step_grad(time, data, rtt)
#         plot_d(ax, grad_step_time, grad_step_data, "b", "Step Gradient")
#         ax.title.set_text(f)
            ax.legend()
            plt.show()
    return time, data, grad_time, grad_data, rtt
#     return time, data 
def get_time_features_bt(curr_file, p):
    main_time, main_data, grad_time, grad_data, rtt= plot_one_bt(curr_file, p)
    
    data = grad_data
    time = grad_time
#     start = math.floor(0.05*len(grad_time))
    mean = data.mean()
#     print("Mean", mean)
    sd = statistics.pstdev(data)
#     print("Std Dev", sd)
    data = abs(data - mean)
    features = []
    tp = 0
    ch_left = 0
    ch_right = 0
    max_win = 0
    min_win = 0
    m = 0.60
    while time[ch_right] - time[ch_left] < rtt :
        max_win = max(max_win, data[ch_right])
        ch_right+=1
    if max_win > m*sd :
        tp = 1

    max_win = 0
    min_win = 0

    ft_left = 0
    left = 0
    right = 0

    while ft_left < len(time) and left < len(time):
        while right < len(time) and (time[right] - time[left]) <= rtt:
            max_win = max(max_win, data[right])
            right+=1
        if tp == 0:
            if (max_win > m*sd):
                features.append([ft_left, left-1, tp])
                ft_left = left
                tp = 1
        if tp == 1:
            if (max_win < m*sd):
                features.append([ft_left, left-1, tp])
                ft_left = left
                tp = 0
    #     print(left, right, max_win, mean + m*sd)
        max_win = 0
        left = right 

    features.append([ft_left, left-1, tp])
    return features, main_time, main_data, grad_time, grad_data, rtt
def get_amp_grad(features, grad_time, grad_data, rtt):
#     mn = grad_data.mean()
#     grad_data -= mn
#     for ft in features :
# #         This idea should be more refined mathematically
#         if ft[2]==1:
#             grad_data[ft[0]:ft[1]+1]*=2
#         if ft[2]==0:
#             grad_data[ft[0]:ft[1]+1]/=2
#     grad_data += mn
#     data = grad_data
#     time = grad_time
#     mean = data.mean()
#     print("Mean", mean)
#     sd = statistics.pstdev(data)
#     print("Std Dev", sd)
#     data = abs(data - mean)
#     features = []
#     tp = 0
#     ch_left = 0
#     ch_right = 0
#     max_win = 0
#     min_win = 0
#     m = 2
#     while time[ch_right] - time[ch_left] < rtt :
#         max_win = max(max_win, data[ch_right])
#         ch_right+=1
#     if max_win > m*sd :
#         tp = 1

#     max_win = 0
#     min_win = 0

#     ft_left = 0
#     left = 0
#     right = 0

#     while ft_left < len(time) and left < len(time):
#         while right < len(time) and (time[right] - time[left]) <= rtt:
#             max_win = max(max_win, data[right])
#             right+=1
#         if tp == 0:
#             if (max_win > m*sd):
#                 features.append([ft_left, left-1, tp])
#                 ft_left = left-1
#                 tp = 1
#         if tp == 1:
#             if (max_win < m*sd):
#                 features.append([ft_left, left-1, tp])
#                 ft_left = left-1
#                 tp = 0
#     #     print(left, right, max_win, mean + m*sd)
#         max_win = 0
#         left = right
        
#     features.append([ft_left, left-1, tp])
    pos = 0
    for ft in features :
        if pos > 0 and pos < len(features)-1 :
            if(round(grad_time[ft[1]] - grad_time[ft[0]],3) <= 3*rtt):
                if features[pos-1][2] == features[pos+1][2]:
                    ft[2] = features[pos-1][2]
#                     model = LinearRegression()
#                     curr_range = list(range(features[pos-1][0],features[pos-1][1]+1))
#                     X = np.reshape(curr_range, (len(curr_range), 1))
#                     model.fit(X, data[features[pos-1][0]:features[pos-1][1]+1])
#                     left = statistics.mean(model.predict(X))
#                     print("Left",left)
#                     curr_range = list(range(features[pos+1][0],features[pos+1][1]+1))
#                     X = np.reshape(curr_range, (len(curr_range), 1))
#                     model.fit(X, data[features[pos+1][0]:features[pos+1][1]+1])
#                     right = statistics.mean(model.predict(X))
#                     print("Right", right)
#                     if left*right < 0 and ft[2] == 0:
#                         ft[2] = 1
#                     if left*right > 0 and ft[2] == 1:
#                         ft[2] = 0            
        pos+=1
    new_ft = []
    start_type = features[0][2]
    start = features[0][0]
    end = 0
    for i in range(len(features)) :
        ft = features[i]
        curr_type = ft[2]
        if curr_type != start_type :
            end = ft[0]
            new_ft.append([start, end, start_type])
            start = end
            start_type = ft[2]
        if i == len(features)-1:
            new_ft.append([start, ft[1], start_type])
        
    return new_ft, grad_time, grad_data
#     return new_ft, grad_time, grad_data
def get_features(features, main_time, grad_time):
    left = 0
    right = 0
    result_ft = []
    for ft in features:
#         print("Next Stop", grad_time[ft[1]])
        while(right < len(main_time) and main_time[right] < grad_time[ft[1]]):
#             print(main_time[right])
            right+=1
        result_ft.append((left, right-1, ft[2]))
        left = right
    if left < right :
        result_ft.append((left, right-1, ft[2]))
    
    return result_ft
color = ['r', 'b']
def get_new_grad(curr_file, p):
    features, main_time, main_data, grad_time, grad_data, rtt = get_time_features_bt(curr_file, p)
    features, grad_time, grad_data = get_amp_grad(features, grad_time, grad_data, rtt)
    main_features = get_features(features, main_time, grad_time)
    if p == 'y' :
        fig, ax = plt.subplots(1,1, figsize=(15,8))
#         plot_d(ax, grad_time, grad_data, "y", "Smoothened Gradient")
        for ft in features : 
    #         print(main_time[ft[1]]-main_time[ft[0]], ft[2])
            ax.plot(main_time[ft[0]:ft[1]+1], main_data[ft[0]:ft[1]+1], color = color[ft[2]])
        plt.show()
    return features, main_time, main_data


In [ ]:
import csv
import matplotlib.pyplot as plt
import sys
import math

SHOW=True
MULTI_GRAPH=False
SMOOTHENING=False
ONLY_STATS=False
s_factor=0.9

'''
TODO: 
o Add functionality where you only plot flows that send more than x bytes of data
o Sort stats and graphs by flow size
o Organize plots by flow size (larger flows have larger graphs)
o Custom smoothening function
'''

fields=["time", "frame_time_rel", "tcp_time_rel", "frame_num", "frame_len", "ip_src", "src_port", "ip_dest", "dest_port", "tcp_len", "seq", "ack"]

class pkt:
    contents=[]
    def __init__(self, fields) -> None:
        self.contents=[]
        for f in fields:
            self.contents.append(f)

    def get(self, field):
        return self.contents[fields.index(field)]
        

def process_flows(cc, dir,t=1,p="y"):
    tcp_tu=244
    name = dir+cc+"-tcp.csv"
    if t==2 :
        name = dir+cc+"-aft-btl-tcp.csv"
    with open(name) as csv_file:
        print("Reading "+name+"...")
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        total_bytes=0
        '''
        Flow tracking:
        o Identify all packets that are either sourced from or headed to 100.64.0.2
        o Group different flows by client's port
        '''
        flows={}
        data_sent=0
        port_set = set()
        for row in csv_reader:
            packet=pkt(row)
            validPkt=False
            if line_count==0:
                # reject the header
                line_count+=1
                continue
            if data_sent == 0 : 
                if "100.64.0." in packet.get("ip_src"):
                    num = int(packet.get("ip_src")[-1])
                    if t==1 and num%2==0:
                        data_sent=1
                        host_port=packet.get("ip_src")
                    elif t==2 and num%2==1:
                        data_sent=1
                        host_port=packet.get("ip_src")
                if "100.64.0." in packet.get("ip_dest"):
                    num = int(packet.get("ip_dest")[-1])
                    if t==1 and num%2==0:
                        data_sent=1
                        host_port=packet.get("ip_dest")
                    elif t==2 and num%2==1:
                        data_sent=1
                        host_port=packet.get("ip_dest")
                if data_sent == 0:
                    continue
                    
            if packet.get("ip_src")==host_port and packet.get("frame_time_rel")!='' and packet.get("ack")!='': 
                # we care about this ACK packet
                validPkt=True
                
                port=packet.get("src_port")
                
                if port not in flows:
                    flows[port]={"loss_win":[],"loss_bif":0,"serverip":packet.get("ip_dest"), "serverport":packet.get("dest_port"), "act_times":[],"times":[], "windows":[], "cwnd":[], "bif":0, "last_ack":0, "last_seq":0, "pif":0, "drop":[], "next":0, "retrans":[]}
                hey = 0
                if int(packet.get("ack"))==flows[port]["last_ack"]:
                    hey = 1
                    flows[port]["loss_bif"]-=tcp_tu
                    if p=="y" : 
                        print(flows[port]["last_ack"])
                
                flows[port]["act_times"].append(packet.get("time"))
                
                flows[port]["times"].append(float(packet.get("frame_time_rel")))
                
                flows[port]["last_ack"]=max(int(flows[port]["last_ack"]),int(packet.get("ack")))
                
                flows[port]["bif"]=flows[port]["last_seq"]-flows[port]["last_ack"]
                
                if hey==1 and p=="y":
                    print("BIF",flows[port]["bif"])
                    
                flows[port]["bif"]=min(flows[port]["loss_bif"],flows[port]["bif"])
                flows[port]["loss_bif"]=flows[port]["bif"]
                if hey==1 and p=="y":
                    print("LOSS",flows[port]["loss_bif"])
                #flows[port]["windows"].append(int(flows[port]["bif"]))
                
                flows[port]["pif"]-=1
                
                flows[port]["cwnd"].append(flows[port]["pif"])
                
                
                
                
            elif packet.get("ip_dest")==host_port and packet.get("frame_time_rel")!='' and packet.get("seq")!='':
                #we care about this Data packet
                validPkt=True
                
                port=packet.get("dest_port")
                
                if port not in flows:
                    flows[port]={"loss_win":[],"loss_bif":0,"serverip":packet.get("ip_src"), "serverport":packet.get("src_port"),"act_times":[], "times":[], "windows":[], "cwnd":[], "bif":0, "last_ack":0, "last_seq":0, "pif":0, "drop":[], "next":0, "retrans":[]}
                
                flows[port]["times"].append(float(packet.get("frame_time_rel")))
                
                flows[port]["act_times"].append(packet.get("time"))
                
                seq = int(packet.get("seq"))
                
                tcp_len = int(packet.get("tcp_len"))
                
                if flows[port]["next"] <  seq :
                    # there has been a packet drop :
                    # update next correctly
                    if p=="y" :
                        print("Packet Drop Next", flows[port]["next"],"Seq",seq)
                    flows[port]["drop"].append(flows[port]["times"][-1])
                    flows[port]["next"] = seq + tcp_len
                
                elif flows[port]["next"] > seq :
                    # this is a retransmission 
                    # dont change the next
                    if p=="y" :
                        print("Packet Retrans Next", flows[port]["next"],"Seq",seq)
                    flows[port]["retrans"].append(flows[port]["times"][-1])
                
                else :
                    # correct order
                    if p=="y":
                        print("Next",flows[port]['next'], "Seq", seq)
                    flows[port]["next"] = seq + tcp_len
                
                add=0
                if int(packet.get("seq")) == flows[port]["last_seq"] + int(packet.get("tcp_len")):
                    add=1
                
                flows[port]["last_seq"]= max(flows[port]["last_seq"],int(packet.get("seq")))
                
                if add==1:
                    flows[port]["bif"] += tcp_len
                
                flows[port]["loss_bif"] = flows[port]["bif"]
                
                flows[port]["pif"]+=1
                
                flows[port]["cwnd"].append(flows[port]["pif"])
            if SMOOTHENING and validPkt and len(flows[port]["windows"])>2:
                flows[port]["windows"].append(int((s_factor*flows[port]["windows"][-1])+((1-s_factor)*flows[port]["bif"])))
            elif validPkt:
                flows[port]["windows"].append(int(flows[port]["bif"]))
                flows[port]["loss_win"].append(int(flows[port]["loss_bif"]))
            line_count+=1
            total_bytes+=int(packet.get("frame_len"))
            #print(line_count, total_bytes)
            
#         print("total bytes processed:", total_bytes/1000, "KBytes for", cc, "(unlimited)")
    return flows

def custom_smooth_function():
    pass

def get_flow_stats(flows):
    num=len(flows.keys())
    print("FLOW STATISTICS: \nNumber of flows: ", num)
    print("------------------------------------------------------------------------------")
    print('%6s'%"port", '%15s'%"SrcIP", '%8s'%"SrcPort",  '%8s'%"duration",  '%8s'%"start",  '%8s'%"end", '%8s'%"Sent (B)", '%8s'%"Recv (B)",)
    for k in flows.keys():
        print('%6s'%k, '%15s'%flows[k]["serverip"], '%8s'%flows[k]["serverport"], '%8s'%str('%.2f'%(flows[k]["times"][-1]-flows[k]["times"][0])), '%8s'%str('%.2f'%flows[k]["times"][0]), '%8s'%str('%.2f'%flows[k]["times"][-1]), '%8s'%flows[k]["last_seq"], '%8s'%flows[k]["last_ack"])
        #print("    * Flow "+str(k)+": ", flows[k]["last_ack"], " ", flows[k]["last_seq"], " bytes transfered.")
    return num

def run(files):
    flows = {}
    for f in files:
        algo_cc=f
        #Get the data for all the flows
        print("==============================================================================")
        print("opening trace ../measurements/"+algo_cc+".csv...")
        flows = process_flows(algo_cc, "./Nebby/measurements/")
        #decide on final graph layout
        num = get_flow_stats(flows)

        if ONLY_STATS:
            sys.exit()

        if num==1:
            MULTI_GRAPH=False
        #grid size
        if MULTI_GRAPH:
            size=(0,0)
            grids={1:(2,2), 2:(2,2), 4:(2,2), 6:(2,3), 9:(3,3), 12:(3,4), 15:(3,5), 16:(4,4), 20:(5,4), 24:(6,4), 30:(6,5), 36:(6,6), 40:(8,5), 42:(8,7), 49:(7,7)}
            g=num
            while g<=49 and g not in grids:
                g+=1
            if g in grids.keys():
                size=grids[g]
            else:
                size=grids[49]  
            fig, axs = plt.subplots(size[0], size[1])
            for i in range(size[0]):
                for j in range(size[1]):
                    #axs[i][j].legend(loc="lower right")
                    if i==size[0]-1:
                        axs[i][j].set_xlabel("Time (s)")
                    if j==0:
                        axs[i][j].set_ylabel("Bytes in flight")
        else:
            plt.xlabel("Time (s)")
            plt.ylabel("Bytes in flight")
        counter=0
        for port in flows.keys():
            if MULTI_GRAPH:  
                axs[counter%size[0]][(counter//size[0])%size[1]].scatter(flows[port]["times"], flows[port]["windows"], color="#858585")
                axs[counter%size[0]][(counter//size[0])%size[1]].plot(flows[port]["times"], flows[port]["windows"], label=str(port), linestyle="solid")
            else:
                plt.plot(flows[port]["times"], flows[port]["windows"], label=str(port), linestyle="solid")
                plt.scatter(flows[port]["times"], flows[port]["windows"], color="#858585")
            counter+=1
        if MULTI_GRAPH:
            counter=0
            for port in flows.keys():
                axs[counter%size[0]][(counter//size[0])%size[1]].legend()
                counter+=1
        else:
            plt.legend()
        if MULTI_GRAPH:
            fig.set_size_inches(16, 12)
        if SHOW:
            plt.show()
        else:
            plt.savefig("../logs/results/"+algo_cc+".png", dpi=600, bbox_inches='tight', pad_inches=0)
    return flows


def get_window(f,p,t=1):
    algo_cc = f
    flows = process_flows(algo_cc, "./Nebby/measurements/",t=t,p=p)
    data = []
    time = []
    drops = []
    retrans = []
    use_port = 0
    maxx = 0
    for port in flows.keys():
        if len(flows[port]['windows']) > maxx:
            maxx = len(flows[port]['windows'])
            use_port = port
    print("Port",use_port)
    data = flows[use_port]['windows']
    time = flows[use_port]['times']
    loss_data = flows[use_port]['loss_win']
    drop = flows[use_port]['drop']
    retrans = flows[use_port]['retrans']
    if p == "p":
        for i in range(len(flows[use_port]['windows'])):
            print("{0:10}{1:10}".format(flows[use_port]['times'][i], flows[use_port]['windows'][i]))
    time_index = len(time)-1
    for index in range(len(flows[use_port]['times'])-1):
        if flows[use_port]['times'][index+1] - flows[use_port]['times'][index] > 1 :
            time_index = index
    time_last = flows[use_port]['times'][time_index]
    data = data[:time_index+1]
    time = time[:time_index+1]
    return data, time, drops, retrans, loss_data